In [1]:
from langchain_community.llms import LlamaCpp

import abc


In [2]:

from langchain_community.llms import LlamaCpp


class LlamaCPP:
    def __init__(self, model_path="models/llama-7b-hf.gguf", n_gpu_layers=41, n_batch=1024, n_ctx=2048) -> None:
        self.model_path = model_path
        self.n_gpu_layers = n_gpu_layers
        self.n_batch = n_batch
        self.n_ctx = n_ctx

        self.model = LlamaCpp(
            model_path=model_path,
            n_gpu_layers=n_gpu_layers,
            n_batch=n_batch,
            n_ctx=n_ctx,
            f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
            verbose=True,
        )
        super().__init__()

    def send_request(self, X, break_word:str = " name:") -> str:
        tok_len = self.model.get_num_tokens(X)
        # print(tok_len)
        if tok_len > 1550:
            raise Exception(f"Request exceeds prefelable 1500 tokens. Has: {tok_len}")
        prev = ""
        res = ""
        for token in self.model.stream(X, echo=False):
            res += token
            if break_word == prev+token:
                # print(res)#dev
                res = res.replace(" ; name:", "")
                res = res.replace(";name:", "")
                res = res.replace(" ;name:", "")
                res = res.replace("; name:", "")
                res = res.replace(" name:", "")
                res = res.replace("name:", "")
                break
            if "\n" in token:
                res = res.replace("\n", "")
                break
            prev = token
        
        res = res.replace("  ;  ", "")
        res = res.replace("  ; ", "")
        res = res.replace("  ;", "")
        res = res.replace(" ; ", "")
        res = res.replace("; ", "")
        res = res.replace(" ;", "")
        res = res.replace(";", "")
        return res

    def get_info(self) -> str:
        return {
            "model":"LLama-13b-hf-q8_0",
            "model_path":self.model_path,
            "n_gpu_layers":self.n_gpu_layers,
            "n_batch":self.n_batch,
            "n_ctx":self.n_ctx,
        }




In [3]:
# test libs

from pprint import pprint
import pandas as pd
import numpy as np
# from transformers import LlamaTokenizerFast

In [4]:
# test request

df = pd.read_csv("small_validation_cbr_00_input.csv")

# req = compose(df.iloc[[11, 15, 112, 122, 133, 144, 155, 8]])
# req = "The second planet in the solar system is "

In [5]:
model = LlamaCPP()

ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   no
ggml_init_cublas: CUDA_USE_TENSOR_CORES: yes
ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4060 Ti, compute capability 8.9, VMM: yes
llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from models/llama-7b-hf.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32   

In [6]:
inputs = df.input.values

In [7]:
for i in range(100):
    res = model.send_request(inputs[i]) # przykład send request
    print(res)
    print("")

Cream butter and sugar until light and creamy. Add egg and beat until combined well. Add coffee granules and beat until combined well. Add flour and beat until combined well. Stir in almonds and powdered sugar and beat until combined well. Form into balls and place on ungreased baking sheet. Bake 350 for 1012 minutes. Makes 24 bars.



Llama.generate: prefix-match hit

llama_print_timings:        load time =     498.93 ms
llama_print_timings:      sample time =      28.86 ms /   170 runs   (    0.17 ms per token,  5891.32 tokens per second)
llama_print_timings: prompt eval time =     553.12 ms /  1141 tokens (    0.48 ms per token,  2062.84 tokens per second)
llama_print_timings:        eval time =    9590.90 ms /   169 runs   (   56.75 ms per token,    17.62 tokens per second)
llama_print_timings:       total time =   10462.26 ms /  1310 tokens
Llama.generate: prefix-match hit


Preheat oven to 350 degrees. Cook sausage in a large non-stick skillet over medium-high heat until brown. Remove with a slotted spoon to a bowldrain and crumble. In the same pan melt butter over medium heat. Add celery and onion and saute 5 minutes. Stir in mushrooms and cook 5 minutes longer. Add the crumbled sausage and cook 1 minute longer. Add the cubed bread and stir to coat. Pour in the chicken broth. Season with salt and pepper. Stir in the sage. Transfer to a greased 13 x 9 baking dish. Bake 20-25 minutes or until bubbly. Serve hot.

Preheat oven to 350f degrees. Grease a 9x13 pan or 13x9 pan with cooking spray. Mix bread cubes with poultry seasoning and salt & pepper to taste. Cut cheese into 8 slices and arrange in a circle around the bread crumbs. Place the slices of turkey in the center of the circle of crumbs & cheese. Pour egg & milk over the top of the bread & cheese mixture. Place pan into the oven. Bake for 45 minutes or until the cheese is melted & the bread is lightl

Llama.generate: prefix-match hit


Bring a large pot of salted water to a boil. Add spaghetti and cook until tender, according to package directions. Drain and rinse with cold water. While pasta is cooking, heat the olive oil in a large pan over medium heat. Add the broccoli florets and cook for 5 minutes. Add the chunky peanut butter and vegetable stock and bring to a boil. Stir in the soy sauce and seasoned rice vinegar. Reduce heat to low and simmer until the sauce thickens. Add the drained pasta and the dried red pepper flakes and stir to coat. Taste and adjust the seasonings with additional salt and red pepper flakes if necessary. Serve immediately.



...

Heat oven to 350 degrees. Grease and flour a 13x9x2 inch pan. Blend yellow cake mix with water and half and half until moistened. Blend in egg and egg substitute. Stir in half of the semi sweet chocolate chips and coconut. Spread mixture into pan. Bake for 35 minutes. While cake is still warm, sprinkle with remaining semi-sweet chips and pecans. Cool. Drizzle with kahlua. Makes about 24 servings.

Position a rack in the upper third of an oven and preheat to 450f. Snap off the tough stem ends from the asparagus spears and arrange the spears on a baking sheet. In a small bowl, whisk together the olive oil, garlic and salt. Brush the asparagus evenly with the oil, turning the spears to coat well. Arrange the almonds around the asparagus. Bake until the asparagus is tender and just turning golden, 6 to 8 minutes. Transfer the asparagus to a warmed serving platter and drizzle with the pan juices.



Llama.generate: prefix-match hit


Cream margarine and flour together. Add milk, sugar and vanilla extract. Heat to boiling point while stirring constantly. Add cooked rice and beat for 2 minutes with an electric beater. Cool slightly before adding the egg yolks. Beat yolks slightly before adding to the mixture. Pour into greased 2 qt baking dish. Bake for 45 minutes at 350f or until set. Serve with a small glass of cold milk.



Llama.generate: prefix-match hit


Preheat the oven to 350 degrees. In a large bowl, combine the breadcrumbs and the poultry seasonings. In another bowl combine the margarine and the flour. Add the salt and pepper to the margarine mixture. Add the seasoned breadcrumbs to the margarine mixture and stir together. Dip each piece of chicken in the margarine mixture, then in the breadcrumbs. Place the breaded chicken in a lightly greased 9 x 13 inch baking pan. Bake for 35-45 minutes or until the juices run clear.



In [8]:
import random
import time

I_ITER = 100


reqs = []
ress = []

start = time.time()
for _ in range(I_ITER):
    req = compose(df.iloc[[random.randint(0, len(df)) for _ in range(8)]])

    res = model.send_request(req) # przykład send request

    reqs.append(req)
    ress.append(res)
    print("----------------------------------------------------------------------------------------------------------------------------------------")
    print(req)
    print(res)
end = time.time()
print((end - start)/I_ITER)


NameError: name 'compose' is not defined

In [ ]:
df = pd.DataFrame()

In [ ]:
ress